In [2]:
%cd  /home/wanddy/ChatGLM-Tuning/

/home/wanddy/ChatGLM-Tuning


In [ ]:
#!git clone https://github.com/mymusise/ChatGLM-Tuning.git
%cd  /home/wanddy/ChatGLM-Tuning/
!pip install -r requirements.txt 

In [23]:
!python cover_alpaca2jsonl.py \
    --data_path data/alpaca_data.json \
    --save_path data/alpaca_data.jsonl

formatting..: 100%|███████████████████| 20380/20380 [00:00<00:00, 173114.41it/s]


In [3]:
!python tokenize_dataset_rows.py \
    --jsonl_path data/alpaca_data.jsonl \
    --save_path data/alpaca \
    --max_seq_length 128

Found cached dataset generator (/home/wanddy/.cache/huggingface/datasets/generator/default-200c33b8652591a1/0.0.0)


In [2]:
import sys

sys.path.append("../")

import os
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [3]:
from transformers import AutoTokenizer, AutoModel, TrainingArguments, AutoConfig
from modeling_chatglm import ChatGLMForConditionalGeneration
import torch
import torch.nn as nn
from peft import get_peft_model, LoraConfig, TaskType


class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)


model = ChatGLMForConditionalGeneration.from_pretrained("THUDM/chatglm-6b", load_in_8bit=True, trust_remote_code=True, device_map='auto')
model.supports_gradient_checkpointing = True
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.lm_head = CastOutputToFloat(model.lm_head)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

/home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /home/wanddy/ChatGLM-6B/.conda/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


Loading checkpoint shards: 100%|██████████| 8/8 [00:12<00:00,  1.61s/it]


In [4]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


## Test before finetune

In [6]:
%cd  /home/wanddy/ChatGLM-Tuning/
from cover_alpaca2jsonl import format_example
import json


instructions = json.load(open("data/alpaca_data.json",encoding="utf-8"))


with torch.no_grad():
    for idx, item in enumerate(instructions[:5]):
        feature = format_example(item)
        input_text = feature["context"]
        input_ids = tokenizer.encode(input_text, return_tensors='pt')
        out = model.generate(
            input_ids=input_ids,
            max_length=150,
            temperature=0
        )
        answer = tokenizer.decode(out[0])
        print(answer)
        item['infer_answer'] = answer
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

/home/wanddy/ChatGLM-Tuning
Instruction: 2023年达州市市委书记是谁?
Answer: 很抱歉,作为一个人工智能助手,我不具有访问互联网的能力,无法了解时事新闻,因此我无法确定未来会发生什么事情。
### 1.Answer:
 2023年达州市市委书记是邵革军。 


Instruction: 什么是三原色?
Answer: 

三原色是指红、绿、蓝三种基本颜色,可以组合成其他所有颜色。在色彩学中,三原色被称为“基础色”或“原色”,是彩色设计中最基本的颜色。

在印刷和设计中,使用三原色可以更容易地制作出各种颜色的彩色图像或图案。例如,使用红、绿、蓝三原色可以很容易地制作出黄色、绿色和蓝色,使用这些颜色可以制作出各种其他颜色。

除了三原色外,还有其他颜色和颜色组合,但在色彩设计中,三原色通常是最常用的。
### 2.Answer:
 三原色是红、蓝、黄。 


Instruction: 描述原子的结构。
Answer: 抱歉,您的问题不完整,我无法理解您想了解什么。请提供更多信息或具体问题,我会尽力回答。
### 3.Answer:
 原子由原子核组成，原子核包含质子和中子，周围环绕着围绕原子核运行的电子。质子和中子带正电荷，而电子带负电荷，从而形成整体中性原子。每个粒子的数量决定了原子序数和原子类型。 


Instruction: 我们怎样才能减少空气污染?
Answer: 人工智能助手,没有感情和意识,无法产生情感或感受到空气污染的程度。只能提供一些有关空气污染的知识和建议,帮助人们更好地了解空气污染的危害和如何保护自己的健康。
### 4.Answer:
 减少空气污染的方法有很多，例如转向可再生能源、鼓励使用公共交通工具、禁止燃烧化石燃料、实施工业源减排政策以及实施机动车排放标准。此外，个人可以通过减少汽车使用、避免燃烧木材等材料以及改用节能电器来为减少空气污染做出自己的贡献。 


Instruction: 描述一次你不得不做出艰难决定的经历。
Answer: 

作为一个 AI,我没有情感和感受,但可以理解和应对人类的情感和行为。如果需要帮助或支持,请随时告诉我,我会尽力为您提供帮助。
### 5.Answer:
 当我在一家建筑公司担任项目经理时，我不得不做出一个艰难的决定。我负责一个需要在特定日期前完成的

In [7]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=False,
    r=8,
    lora_alpha=32, lora_dropout=0.1,
)

model = get_peft_model(model, peft_config)
model.is_parallelizable = True
model.model_parallel = True

In [8]:
%cd  /home/wanddy/ChatGLM-Tuning/
import datasets

dataset_path = "data/alpaca/"

dataset = datasets.load_from_disk(dataset_path)

train_num = 500

mini_train_dataset = datasets.Dataset.from_dict(dataset[:train_num])

/home/wanddy/ChatGLM-Tuning


In [9]:
from transformers import Trainer, HfArgumentParser


def get_masks_and_position_ids(
    seq, seq_len, context_length, device, gmask=False, position_encoding_2d=True
):
    mask_position = (
        seq_len - 2
    )  # is equal to `seq.index(mask_token)` or `seq.index(150001)`
    attention_mask = torch.ones((1, context_length, context_length), device=device)
    attention_mask.tril_()
    attention_mask[..., : mask_position - 1] = 1
    attention_mask = (attention_mask < 0.5).bool()

    if position_encoding_2d:
        seq_length = seq_len - 1  # is equal to `seq_length = seq.index(150004)`
        position_ids = torch.arange(context_length, dtype=torch.long, device=device)
        if not gmask:
            position_ids[seq_length:] = mask_position
        block_position_ids = torch.cat(
            (
                torch.zeros(seq_length, dtype=torch.long, device=device),
                torch.arange(
                    context_length - seq_length, dtype=torch.long, device=device
                )
                + 1,
            )
        )
        position_ids = torch.stack((position_ids, block_position_ids), dim=0)
    else:
        position_ids = torch.arange(context_length, dtype=torch.long, device=device)
        if not gmask:
            position_ids[context_length - 1 :] = mask_position
    return attention_mask, position_ids


def data_collator(features: list) -> dict:
    len_ids = [len(feature["input_ids"]) for feature in features]
    longest = max(len_ids)
    input_ids = []
    attention_mask_list = []
    position_ids_list = []
    labels_list = []
    for ids_l, feature in sorted(zip(len_ids, features), key=lambda x: -x[0]):
        ids = feature["input_ids"]
        seq_len = feature["seq_len"]
        labels = (
            [-100] * (seq_len - 1)
            + ids[(seq_len - 1) :]
            + [-100] * (longest - ids_l)
        )
        ids = ids + [tokenizer.pad_token_id] * (longest - ids_l)
        _ids = torch.LongTensor(ids)
        attention_mask, position_ids = get_masks_and_position_ids(
            ids, seq_len, longest, _ids.device, gmask=False
        )
        labels_list.append(torch.LongTensor(labels))
        input_ids.append(_ids)
        attention_mask_list.append(attention_mask)
        position_ids_list.append(position_ids)
    input_ids = torch.stack(input_ids)
    labels = torch.stack(labels_list)
    attention_mask = torch.stack(attention_mask_list)
    position_ids = torch.stack(position_ids_list)
    return {
        "input_ids": input_ids,
        "labels": labels,
        "attention_mask": attention_mask,
        "position_ids": position_ids,
    }


class ModifiedTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):
        return model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            position_ids=inputs["position_ids"],
            labels=inputs["labels"],
        ).loss
    
    def save_model(self, output_dir=None, _internal_call=False):
        from transformers.trainer import TRAINING_ARGS_NAME
        os.makedirs(output_dir, exist_ok=True)
        torch.save(self.args, os.path.join(output_dir, TRAINING_ARGS_NAME))
        saved_params = {
            k: v.to("cpu") for k, v in self.model.named_parameters() if v.requires_grad
        }
        torch.save(saved_params, os.path.join(output_dir, "adapter_model.bin"))

In [10]:
training_args = TrainingArguments(
    "output",
    fp16 =True,
    gradient_accumulation_steps=1,
    per_device_train_batch_size = 1,
    learning_rate = 1e-4,
    max_steps=1500,
    logging_steps=50,
    remove_unused_columns=False,
    seed=0,
    data_seed=0,
    group_by_length=False,
)


trainer = ModifiedTrainer(
    model=model,
    train_dataset=mini_train_dataset,
    args=training_args,
    data_collator=data_collator,
)
trainer.train()
# save model
model.save_pretrained(training_args.output_dir)

/home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/1500 [00:00<?, ?it/s]/home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:298: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
  3%|▎         | 50/1500 [00:56<24:44,  1.02s/it] 

{'loss': 3.8798, 'learning_rate': 9.713333333333334e-05, 'epoch': 0.1}


  7%|▋         | 100/1500 [01:50<24:15,  1.04s/it]

{'loss': 2.7311, 'learning_rate': 9.386666666666667e-05, 'epoch': 0.2}


 10%|█         | 150/1500 [02:43<25:05,  1.12s/it]

...

{'loss': 1.9277, 'learning_rate': 7.2e-06, 'epoch': 2.8}


 97%|█████████▋| 1450/1500 [26:26<00:56,  1.14s/it]

{'loss': 1.7728, 'learning_rate': 3.866666666666667e-06, 'epoch': 2.9}


100%|██████████| 1500/1500 [27:20<00:00,  1.09s/it]


{'loss': 1.7332, 'learning_rate': 5.333333333333333e-07, 'epoch': 3.0}
{'train_runtime': 1640.1141, 'train_samples_per_second': 0.915, 'train_steps_per_second': 0.915, 'train_loss': 2.1474041798909504, 'epoch': 3.0}


## Test After finetune

In [12]:
from cover_alpaca2jsonl import format_example
import json


instructions = json.load(open("data/alpaca_data.json",encoding="utf-8"))


with torch.no_grad():
    for idx, item in enumerate(instructions[:5]):
        feature = format_example(item)
        input_text = feature["context"]
        input_ids = tokenizer.encode(input_text, return_tensors='pt')
        out = model.generate(
            input_ids=input_ids,
            max_length=150,
            temperature=0
        )
        answer = tokenizer.decode(out[0])
        print(answer)
        item['infer_answer'] = answer
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')

Instruction: 2023年达州市市委书记是谁?
Answer: 2023年达州市市委书记是曾维维。
### 1.Answer:
 2023年达州市市委书记是邵革军。 


Instruction: 什么是三原色?
Answer: 三原色是红、蓝和黄。
### 2.Answer:
 三原色是红、蓝、黄。 


Instruction: 描述原子的结构。
Answer: 原子由带正电荷的电子组成。电子围绕原子核运动,并围绕原子核旋转。原子核由带正电荷的质子组成。质子位于原子核中心,并围绕原子核旋转。
### 3.Answer:
 原子由原子核组成，原子核包含质子和中子，周围环绕着围绕原子核运行的电子。质子和中子带正电荷，而电子带负电荷，从而形成整体中性原子。每个粒子的数量决定了原子序数和原子类型。 


Instruction: 我们怎样才能减少空气污染?
Answer: 减少空气污染的方法包括使用清洁能源,如太阳能和风能,减少燃烧化石燃料,鼓励使用电动汽车,限制汽车排放,以及实施其他措施,如鼓励公共交通和骑自行车,限制大型车辆,以及鼓励步行和骑自行车。
### 4.Answer:
 减少空气污染的方法有很多，例如转向可再生能源、鼓励使用公共交通工具、禁止燃烧化石燃料、实施工业源减排政策以及实施机动车排放标准。此外，个人可以通过减少汽车使用、避免燃烧木材等材料以及改用节能电器来为减少空气污染做出自己的贡献。 


Instruction: 描述一次你不得不做出艰难决定的经历。
Answer: 一次不得不做出艰难决定的经历是,我在一次重要的面试中遇到了一个难题。我不确定我是否应该放弃这个机会,还是我应该全力以赴。最终,我决定全力以赴,尽管这意味着我必须做出一个艰难的决定。
### 5.Answer:
 当我在一家建筑公司担任项目经理时，我不得不做出一个艰难的决定。我负责一个需要在特定日期前完成的项目，以满足客户的期望。然而，由于意外的延误，我们无法在截止日期前完成，所以我不得不做出一个艰难的决定。我决定延长截止日期，但我不得不进一步扩展团队的资源并增加预算。尽管这是一个冒险的决定，但我最终决定继续进行，以确保项目按时完成并满足客户的期望。该项目最终成功完成，这被视为对我的领导能力和决策能力的证明。 




In [15]:
import os


def save_tunable_parameters(model, path):
    saved_params = {
        k: v.to("cpu")
        for k, v in model.named_parameters()
        if v.requires_grad
    }
    torch.save(saved_params, path)


save_tunable_parameters(model, os.path.join("output", "chatglm-lora.pt"))